In [1]:
import urllib3
import json
import random
import time
import os
import logging
import requests

import numpy as np
import pandas as pd

In [2]:
to_save_path = '../data/'

# Crawl Documents

In this notebook we show how we extracted the documents from [Regione Toscana](http://www.regione.toscana.it/bancadati/atti/)

## Get Typology Documents

In [3]:
url = 'http://www.regione.toscana.it/bancadati/atti/getTipologiaDelibere.json'

r = requests.get(url)

In [4]:
df_type_delibere = pd.DataFrame(r.json()['getTipologiaDelibere']['Tipologia'])
df_type_delibere

,ID,NOME
0,DG,
1,P07,COMUNICAZIONE
2,P05,DECISIONE
3,P06,DECISIONE DI CONTROLLO
4,DEL,DELIBERA
5,P09,DOCUMENTO PRELIMINARE
6,P03,PROPOSTA DI DELIBERAZIONE AL C.R.
7,P01,PROPOSTA DI LEGGE
8,P02,PROPOSTA DI REGOLAMENTO
9,P04,PROPOSTA DI RISOLUZIONE AL C.R.


In [5]:
df_type_delibere.to_csv(to_save_path + "tipologia_delibere.csv", index=False)
r.close()

## Get The Name of Dirigential Offices

In [6]:
url = 'http://www.regione.toscana.it/bancadati/atti/getDg.json'
r = requests.get(url)

In [7]:
df_struttura = pd.DataFrame(r.json()['getDg']['Struttura'])
df_struttura.sort_values(by=['ID'])

,ID,NOME
10,01025,DIPARTIMENTO ORGANIZZAZIONE ...
9,01027,DIPARTIMENTO BILANCIO E FINANZE ...
1,01923,AVVOCATURA REGIONALE ...
14,01928,DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E G...
11,01931,DIPARTIMENTO ORGANIZZAZIONE E RISORSE ...
16,01934,DIPARTIMENTO SVILUPPO ECONOMICO ...
13,01937,DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTA...
15,01943,DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA ...
12,01946,DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTUR...
42,02090,UFFICI DEL GENIO CIVILE ...


In [8]:
df_struttura.to_csv(to_save_path + 'dg_struttura.csv', index=False)

## Download "Atti Giunta"

parameters and base url

In [3]:
act_types = {
    'atti_dirigenti': 'DMON', 
    'atti_presidente': 'DPG',
    'atti_giunta': 'DAD'
}

year = 2017
start = 0
num = 200
from_number = 0
to_number = 200

base_url = "http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start={:d}&num={:d}&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3{}+AND+inmeta%3ADATA_ATTO%3A{}-01-01..{}-12-31+AND+inmeta%3ANUMERO%3A{}..{}+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AA"

In [4]:
example_url = base_url.format(start, num, act_types['atti_giunta'], year, year, from_number, to_number)
example_url

'http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=0&num=200&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DAD+AND+inmeta%3ADATA_ATTO%3A2017-01-01..2017-12-31+AND+inmeta%3ANUMERO%3A0..200+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AA'

In [5]:
r = requests.get(example_url)

In [6]:
r.status_code

200

In [7]:
data = r.json()

In [8]:
def navigate_json_path(json, path):
    for e in path:
        json = json[e]
    return json

In [9]:
def extract_data(json_data, json_path=['GSP', 'RES', 'R']):
    data = navigate_json_path(json_data, json_path)
    results = []
    for row in data:
        results.append(row['MT'])
    return results

In [10]:
def unwrap_list(x):
    if type(x) == list:
        return x[0]
    else:
        return x

In [11]:
results = extract_data(data)
df_giunta = pd.DataFrame(results)

df_giunta = df_giunta.applymap(unwrap_list)
df_giunta.head()

,ALLEGATO_DESCRIZIONE,ALLEGATO_ID,ALLEGATO_TIPO_PUBBL,CODICE_PRATICA,Creator,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,DT_UTENTE,...,OGGETTO,PERSONA,STATUS,TESTO_RISPOSTA,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,creation date,publisher,revision date
0,void,0,void,201701100000012016DG00000001750,NaN,2017-01-10,Atto soggetto a pubblicazione integrale (PBURT...,Delibera,Atto della giunta,2017-01-12 14:10:26.851966,...,Procedimento di verifica di Assoggettabilità S...,140,2,Delibera n. 1 del 10/01/2017,00000,50118,04788,D:20170112141020+01'00',iText 2.1.7 by 1T3XT,D:20170112141020+01'00'
1,void,0,void,201701100000012016DG00000001765,NaN,2017-01-10,Atto soggetto a pubblicazione su Banca Dati (PBD),Documento preliminare,Atto della giunta,2017-01-12 14:19:38.101904,...,Piano di Tutela delle Acque della Toscana (ex ...,1404,2,Documento preliminare n. 1 del 10/01/2017,00000,50119,05121,D:20170112141937+01'00',iText 2.1.7 by 1T3XT,D:20170112141937+01'00'
2,void,0,void,201701100000022016DG00000001712,NaN,2017-01-10,Atto soggetto a pubblicazione su Banca Dati (PBD),Proposta di legge,Atto della giunta,2017-01-12 12:59:18.461721,...,Attività europee e di rilievo internazionale d...,140#@#1408,2,Proposta di legge n. 2 del 10/01/2017,00000,50112,02002,D:20170112125916+01'00',iText 2.1.7 by 1T3XT,D:20170112125916+01'00'
3,void,0,void,201701100000022016DG00000001751,NaN,2017-01-10,Atto soggetto a pubblicazione su Banca Dati (PBD),Delibera,Atto della giunta,2017-01-17 10:42:31.904961,...,Impugnativa alla Corte Costituzionale del Decr...,140,2,Delibera n. 2 del 10/01/2017,00000,50007,00000,D:20170117104213+01'00',iText 2.1.7 by 1T3XT,D:20170117104213+01'00'
4,void,0,void,201701100000032016DG00000001731,NaN,2017-01-10,Atto soggetto a pubblicazione integrale (PBURT...,Delibera,Atto della giunta,2017-01-12 14:11:44.029312,...,"Approvazione e formalizzazione, ai sensi e per...",1401,2,Delibera n. 3 del 10/01/2017,00000,50113,02511,D:20170112141141+01'00',iText 2.1.7 by 1T3XT,D:20170112141141+01'00'


## Download Atti Presidente

In [12]:
example_url = base_url.format(start, num, act_types['atti_presidente'], year, year, from_number, to_number)
example_url

'http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=0&num=200&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DPG+AND+inmeta%3ADATA_ATTO%3A2017-01-01..2017-12-31+AND+inmeta%3ANUMERO%3A0..200+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AA'

In [13]:
r = requests.get(example_url)
data = r.json()

In [14]:
results = extract_data(data)    

df_atti = pd.DataFrame(results)

df_atti = df_atti.applymap(unwrap_list)
df_atti.head()

,ALLEGATO_DESCRIZIONE,ALLEGATO_ID,ALLEGATO_TIPO_PUBBL,CODICE_PRATICA,Creator,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,DT_UTENTE,...,OGGETTO,PERSONA,STATUS,TESTO_RISPOSTA,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,creation date,publisher,revision date
0,void,0,void,20170000012017AP0013,NaN,2017-02-21,Atto soggetto a pubblicazione integrale (PBURT...,Ordinanza del Presidente,Atto del presidente,2017-02-21 11:49:34.865008,...,Art. 191 del Dlgs 152/06. Discarica in localit...,void,2,Ordinanza del Presidente n. 1 del 21/02/2017,00000,50118,05108,D:20170221114933+01'00',iText 2.1.7 by 1T3XT,D:20170221114933+01'00'
1,void,0,void,20170000012017AP0208,NaN,2017-01-11,Atto soggetto a pubblicazione integrale (PBURT...,Decreto del Presidente,Atto del presidente,2017-01-11 16:05:21.09736,...,Commissione di concorso per n. 1 posto di Diri...,void,2,Decreto del Presidente n. 1 del 11/01/2017,00000,50113,00000,D:20170111160519+01'00',iText 2.1.7 by 1T3XT,D:20170111160519+01'00'
2,void,0,void,20170000022017AP0001,NaN,2017-01-16,Atto soggetto a pubblicazione integrale (PBURT...,Decreto del Presidente,Atto del presidente,2017-01-16 08:59:10.325308,...,COMMISSIONE DI DEGUSTAZIONE VINI TCA - PRATO. ...,void,2,Decreto del Presidente n. 2 del 16/01/2017,00000,50113,00000,D:20170116085907+01'00',iText 2.1.7 by 1T3XT,D:20170116085907+01'00'
3,void,0,void,20170000022017AP0084,NaN,2017-06-29,Atto soggetto a pubblicazione integrale (PBURT...,Ordinanza del Presidente,Atto del presidente,2017-06-29 12:02:03.974215,...,Reiterazione dell'Ordinanza contingibile ed ur...,void,2,Ordinanza del Presidente n. 2 del 29/06/2017,00000,50118,05866,D:20170629120201+02'00',iText 2.1.7 by 1T3XT,D:20170629120201+02'00'
4,void,0,void,20170000032017AP0149,NaN,2017-10-02,Atto soggetto a pubblicazione integrale (PBURT...,Ordinanza del Presidente,Atto del presidente,2017-10-02 15:18:03.09593,...,Reiterazione dell'ordinanza contingibile ed ur...,void,2,Ordinanza del Presidente n. 3 del 02/10/2017,00000,50118,05984,D:20171002151801+02'00',iText 2.1.7 by 1T3XT,D:20171002151801+02'00'


## Download Atti Dei Dirigenti

In [15]:
example_url = base_url.format(start, num, act_types['atti_dirigenti'], year, year, from_number, to_number)
example_url

'http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=0&num=200&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3ADATA_ATTO%3A2017-01-01..2017-12-31+AND+inmeta%3ANUMERO%3A0..200+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AA'

In [16]:
r = requests.get(example_url)
data = r.json()

In [17]:
results = extract_data(data)    

df_dirigenti = pd.DataFrame(results)

df_dirigenti = df_dirigenti.applymap(unwrap_list)
df_dirigenti.head()

,ALLEGATO_DESCRIZIONE,ALLEGATO_ID,ALLEGATO_TIPO_PUBBL,CODICE_PRATICA,Creator,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,DT_UTENTE,...,OGGETTO,PERSONA,STATUS,TESTO_RISPOSTA,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,creation date,publisher,revision date
0,void,0,void,20170000012016AD00000017891,NaN,2017-01-02,Atto pubblicato su Banca Dati (PBD),Decreto non soggetto a controllo interno,Atto dei dirigenti,2017-01-02 11:34:32.955471,...,D.lgs. 150/2012 - Piano nazionale per l'uso so...,005595,2,Decreto n. 1 del 02/01/2017,00000,50117,05015,D:20170102102315+01'00',iText 2.1.7 by 1T3XT; modified using iText® 5....,D:20170102113149+01'00'
1,void,0,void,20170000022016AD00000017976,NaN,2017-01-02,Atto pubblicato su Banca Dati (PBD),Decreto non soggetto a controllo interno,Atto dei dirigenti,2017-01-02 11:43:31.001254,...,"VEZZOSI MAURO E FILIPPO S.n.c. , stabilimento ...",018285,2,Decreto n. 2 del 02/01/2017,00000,50118,05106,D:20170102110602+01'00',iText 2.1.7 by 1T3XT; modified using iText® 5....,D:20170102114220+01'00'
2,void,0,void,20170000032016AD00000017981,NaN,2017-01-02,Atto pubblicato su Banca Dati (PBD),Decreto non soggetto a controllo interno,Atto dei dirigenti,2017-01-02 11:51:51.747645,...,Approvazione di verbale integrativo al verbale...,018491,2,Decreto n. 3 del 02/01/2017,00000,50115,04196,D:20170102104836+01'00',iText 2.1.7 by 1T3XT; modified using iText® 5....,D:20170102115042+01'00'
3,void,0,void,20170000042016AD00000017962,NaN,2017-01-02,Atto pubblicato su Banca Dati (PBD),Decreto non soggetto a controllo interno,Atto dei dirigenti,2017-01-02 12:17:27.972291,...,F.lli Gori di Gori Giovanni & Mario s.n.c. (AR...,018285,2,Decreto n. 4 del 02/01/2017,00000,50118,05106,D:20170102104842+01'00',iText 2.1.7 by 1T3XT; modified using iText® 5....,D:20170102121505+01'00'
4,void,0,void,20170000052017AD00000000003,NaN,2017-01-02,Atto pubblicato su Banca Dati (PBD),Decreto non soggetto a controllo interno,Atto dei dirigenti,2017-01-02 12:20:28.627011,...,"Ditta Morescalchi Srl, attività di officina me...",018285,2,Decreto n. 5 del 02/01/2017,00000,50118,05106,D:20170102111011+01'00',iText 2.1.7 by 1T3XT; modified using iText® 5....,D:20170102122007+01'00'
